In [1]:
#import
import pandas as pd
import numpy as np

#PATH dove andare a leggere e a scrivere
PATH_TO_WRITE_ETL = "./dataset_modificato_WC/"
PATH_FROM_READ_ETL = "./dataset_originale_WC/"
PATH_FROM_READ_ANALYSIS = "./dataset_modificato_WC/" 

In [2]:
#INIZIO RAGGRUPPAMENTO DATI PER INIZIARE L'ANALISI SULLE NAZIONI


df_team_appearances = pd.read_csv(PATH_FROM_READ_ANALYSIS + "team_appearances.csv")
df_bookings = pd.read_csv(PATH_FROM_READ_ANALYSIS + "bookings.csv")
df_goals = pd.read_csv(PATH_FROM_READ_ANALYSIS + "goals.csv")
df_group_standings = pd.read_csv(PATH_FROM_READ_ANALYSIS + "group_standings.csv")
df_matches = pd.read_csv(PATH_FROM_READ_ANALYSIS + "matches.csv")
df_squads = pd.read_csv(PATH_FROM_READ_ANALYSIS + "squads.csv")
df_stadiums = pd.read_csv(PATH_FROM_READ_ANALYSIS + "stadiums.csv")
df_substitution = pd.read_csv(PATH_FROM_READ_ANALYSIS + "substitutions.csv")
df_tournaments = pd.read_csv(PATH_FROM_READ_ANALYSIS + "tournaments.csv")
df_tournament_stages = pd.read_csv(PATH_FROM_READ_ANALYSIS + "tournament_stages.csv")
df_tournament_standings = pd.read_csv(PATH_FROM_READ_ANALYSIS + "tournament_standings.csv")
df_menagers_appointments = pd.read_csv(PATH_FROM_READ_ANALYSIS + "manager_appointments.csv")
df_player = pd.read_csv(PATH_FROM_READ_ANALYSIS + "player_appearances.csv")

In [3]:
#CALCOLO CARTELLINI GIALLI, ROSSI ED ESPULSIONI PER OGNI PARTITA

team_data = df_team_appearances.groupby(["team_id"])[['team_name','team_code','goal_differential','win','lose']].agg('sum').reset_index().sort_values(by="team_id",ascending=True)

team_data['win'] = team_data['win'] - team_data['lose']
team_data = team_data.rename(columns={"win": "win_lose_differential"})
team_data = team_data.drop(columns=["lose"])
team_data = team_data.replace('T-82', 'T-29')
booking_team_data = df_bookings.groupby(["team_id"])[['team_name','team_code','yellow_card','second_yellow_card','red_card']].agg('sum').reset_index().sort_values(by="team_id",ascending=True)
booking_team_data = booking_team_data.replace('T-82', 'T-29')
team_data = team_data.merge(booking_team_data, on="team_id", how="left")
team_data['yellow_card'] = team_data['yellow_card'].fillna(0).astype(int)
team_data['second_yellow_card'] = team_data['second_yellow_card'].fillna(0).astype(int)
team_data['red_card'] = team_data['red_card'].fillna(0).astype(int)

team_data['red_card'] = team_data['red_card'] + team_data['second_yellow_card']
team_data['yellow_card'] = team_data['yellow_card'] + team_data['second_yellow_card']

team_data = team_data.drop(columns=["second_yellow_card"])


C:\Users\mat98\AppData\Local\Temp\ipykernel_26060\1709635747.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  team_data = df_team_appearances.groupby(["team_id"])[['team_name','team_code','goal_differential','win','lose']].agg('sum').reset_index().sort_values(by="team_id",ascending=True)
C:\Users\mat98\AppData\Local\Temp\ipykernel_26060\1709635747.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  booking_team_data = df_bookings.groupby(["team_id"])[['team_name','team_code','yellow_card','second_yellow_card','red_card']].agg('sum').reset_index().sort_values(by="team_id",ascending=True)


In [4]:
#CALCOLO GIOCATORI PER OGNI SQUADRA

players = df_player[['team_id','player_id']].drop_duplicates()
players = players.replace('T-82', 'T-29')
players = players.groupby(["team_id"])[['player_id']].agg('count').reset_index().sort_values(by="team_id",ascending=True)

team_data = team_data.merge(players, on="team_id", how="left")
team_data = team_data.rename(columns={"player_id": "player_per_teams"})


In [5]:
# CALCOLO EDIZIONI A CUI HA PARTECIPATO OGNI SQUADRA
world_cup_appearence = df_player[['team_id','tournament_id']].drop_duplicates()
world_cup_appearence = world_cup_appearence.replace('T-82', 'T-29')
world_cup_appearence = world_cup_appearence.groupby(["team_id"])[['tournament_id']].agg('count').reset_index().sort_values(by="team_id",ascending=True)
team_data = team_data.merge(world_cup_appearence, on="team_id", how="left")
team_data = team_data.rename(columns={"tournament_id": "world_cup_appearence"})

In [6]:
# CALCOLO FINALI A CUI HANNO PARTECIPATO E HANNO VINTO

final_stage = df_player[['team_id','tournament_id','stage_name']].drop_duplicates()
final_stage = final_stage.replace('final round', 'final')
final_stage = final_stage.replace('T-82', 'T-29')
final_stage = final_stage[final_stage.stage_name.eq('final')].reset_index().drop(columns='index',axis=1)
final_stage = final_stage.groupby(["team_id"])[['stage_name']].agg('count').reset_index().sort_values(by="team_id",ascending=True)


team_data = team_data.merge(final_stage, on="team_id", how="left")
team_data['stage_name'] = team_data['stage_name'].fillna(0).astype(int)
team_data = team_data.rename(columns={"stage_name": "finals_reached"})



final_win = df_tournament_standings[['team_id','position']]
final_win = final_win[final_win.position==1].reset_index().drop(columns='index',axis=1)
final_win = final_win.replace('T-82', 'T-29')
final_win = final_win.groupby(["team_id"])[['position']].agg('sum').reset_index().sort_values(by="team_id",ascending=True)
team_data = team_data.merge(final_win, on="team_id", how="left")
team_data['position'] = team_data['position'].fillna(0).astype(int)
team_data = team_data.rename(columns={"position": "finals_win"})


team_data


,team_id,goal_differential,win_lose_differential,yellow_card,red_card,player_per_teams,world_cup_appearence,finals_reached,finals_win
0,T-01,-6,-4,19,2,57,4,0,0
1,T-02,-1,-1,12,1,17,1,0,0
2,T-03,44,23,122,7,221,17,5,2
3,T-04,-18,-8,33,4,63,5,0,0
4,T-05,-4,-1,23,1,93,7,0,0
...,...,...,...,...,...,...,...,...,...
81,T-81,0,0,0,0,14,1,0,0
82,T-29,54,27,81,5,235,19,8,4
83,T-29,54,27,35,3,235,19,8,4
84,T-83,14,2,30,2,115,9,0,0


In [7]:
# CALCOLO ALLENATORI DIVERSI CHE HANNO ALLENATO QUELLA SQUADRA

managers = df_menagers_appointments[['team_id','manager_id']].drop_duplicates()
managers = managers.replace('T-82', 'T-29')
managers = managers.groupby(["team_id"])[['manager_id']].agg('count').reset_index().sort_values(by="team_id",ascending=True)
team_data = team_data.merge(managers, on="team_id", how="left")
team_data = team_data.rename(columns={"manager_id": "different_managers"})

team_data

,team_id,goal_differential,win_lose_differential,yellow_card,red_card,player_per_teams,world_cup_appearence,finals_reached,finals_win,different_managers
0,T-01,-6,-4,19,2,57,4,0,0,4
1,T-02,-1,-1,12,1,17,1,0,0,1
2,T-03,44,23,122,7,221,17,5,2,15
3,T-04,-18,-8,33,4,63,5,0,0,5
4,T-05,-4,-1,23,1,93,7,0,0,8
...,...,...,...,...,...,...,...,...,...,...
81,T-81,0,0,0,0,14,1,0,0,1
82,T-29,54,27,81,5,235,19,8,4,10
83,T-29,54,27,35,3,235,19,8,4,10
84,T-83,14,2,30,2,115,9,0,0,8


In [8]:
# TOLGO RIGHE IN ECCESSO

team_data = team_data.drop(index=29)
team_data = team_data.drop(index=82)
team_data = team_data.drop(index=83)

In [9]:
# AGGIUNGO GOAL FATTI

team_goal = df_team_appearances[['team_id','goals_for','goals_against']]
team_goal = team_goal.replace('T-82', 'T-29')
team_goal = df_team_appearances.groupby(["team_id"])[['goals_for','goals_against']].agg('sum').reset_index().sort_values(by="team_id",ascending=True)
team_data = team_data.merge(team_goal, on="team_id", how="left")

team_name = df_team_appearances[['team_id','team_name']].drop_duplicates()
team_data = team_data.merge(team_name, on="team_id", how="left")

team_data


,team_id,goal_differential,win_lose_differential,yellow_card,red_card,player_per_teams,world_cup_appearence,finals_reached,finals_win,different_managers,goals_for,goals_against,team_name
0,T-01,-6,-4,19,2,57,4,0,0,4,13,19,Algeria
1,T-02,-1,-1,12,1,17,1,0,0,1,1,2,Angola
2,T-03,44,23,122,7,221,17,5,2,15,137,93,Argentina
3,T-04,-18,-8,33,4,63,5,0,0,5,13,31,Australia
4,T-05,-4,-1,23,1,93,7,0,0,8,43,47,Austria
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,T-79,-25,-11,54,4,114,10,0,0,9,37,62,United States
79,T-80,13,5,68,9,168,13,2,2,9,87,74,Uruguay
80,T-81,0,0,0,0,14,1,0,0,1,4,4,Wales
81,T-83,14,2,30,2,115,9,0,0,8,60,46,Yugoslavia


In [10]:
# TRASFORMO IN CSV

team_data.to_csv("./stats/stats_team.csv",index=False)

In [33]:
team_per_year_data = df_team_appearances[['tournament_id','team_name','goals_for','goals_against','win','lose']]
team_per_year_data = team_per_year_data.replace('West Germany', 'Germany')
team_per_year_data = team_per_year_data.replace('East Germany', 'Germany')
team_per_year_data = team_per_year_data.groupby(["tournament_id",'team_name'])[['goals_for','goals_against','win','lose']].agg('sum').reset_index().sort_values(by="tournament_id",ascending=True)

booking_team_per_year_data = df_bookings[['tournament_id','team_name','yellow_card','second_yellow_card','red_card']]
booking_team_per_year_data = booking_team_per_year_data.replace('West Germany', 'Germany')
booking_team_per_year_data = booking_team_per_year_data.replace('East Germany', 'Germany')
booking_team_per_year_data = booking_team_per_year_data.groupby(["tournament_id",'team_name'])[['yellow_card','second_yellow_card','red_card']].agg('sum').reset_index().sort_values(by="tournament_id",ascending=True)
team_per_year_data = team_per_year_data.merge(booking_team_per_year_data, on=["tournament_id",'team_name'], how="left")
team_per_year_data['yellow_card'] = team_per_year_data['yellow_card'].fillna(0).astype(int)
team_per_year_data['second_yellow_card'] = team_per_year_data['second_yellow_card'].fillna(0).astype(int)
team_per_year_data['red_card'] = team_per_year_data['red_card'].fillna(0).astype(int)

team_per_year_data['red_card'] = team_per_year_data['red_card'] + team_per_year_data['second_yellow_card']
team_per_year_data['yellow_card'] = team_per_year_data['yellow_card'] + team_per_year_data['second_yellow_card']

team_per_year_data = team_per_year_data.drop(columns=["second_yellow_card"])
team_per_year_data['tournament_id'] = team_per_year_data['tournament_id'].str.slice(3,7)
team_per_year_data = team_per_year_data.rename(columns={"tournament_id": "tournament_year"})
#booking_team_per_year_data
team_per_year_data
team_per_year_data.to_csv("./stats/stats_team_per_year.csv",index=False)